In [ ]:
# Instalar las bibliotecas necesarias
!pip install PyMuPDF pandas openpyxl

import fitz  # PyMuPDF
import os
import pandas as pd
import re

# Ruta a la carpeta que contiene las resoluciones directorales en Google Drive
folder_path = '/content/prueba'
vessels_path = '/content/national_vessels.xlsx'

# Cargar el archivo de embarcaciones pesqueras
vessels_df = pd.read_excel(vessels_path)
vessels_dict = vessels_df.set_index('MATRICULA').T.to_dict('dict')

# Lista de procedimientos TUPA y palabras clave asociadas
procedimientos_tupa = {
    "AIF": ["incremento de flota", "construcción", "adquisición de embarcaciones"],
    "AMPLIACION PLAZO AIF": ["ampliación de plazo", "autorización de incremento de flota"],
    "CAMBIO TITULARIDAD AIF": ["cambio de titularidad", "incremento de flota otorgada"],
    "PERMISO EP NACIONAL": ["permiso de pesca", "embarcaciones de menor y mayor escala de bandera nacional"],
    "AMPLIACION PERMISO VIA IF": ["ampliación de permiso", "incremento de flota autorizado"],
    # Añade más procedimientos y palabras clave aquí
    # ...
}

# Función para extraer los artículos y el punto 1 de CONSIDERANDO de un PDF
def extract_articles_from_pdf(pdf_path):
    # Abrir el archivo PDF
    doc = fitz.open(pdf_path)
    articles = []
    inside_se_resuelve = False
    current_article = ""

    # Variable para almacenar el punto 1 de CONSIDERANDO si se encuentra
    considerando_punto_1 = ""
    frase_tupa = ""

    # Recorrer cada página del PDF y obtener el texto completo
    all_text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        all_text += page.get_text("text") + "\n"

    # Extraer el punto 1 de la sección CONSIDERANDO
    considerando_match = re.search(r'CONSIDERANDO:\s*(1\..*?)(?=\n\n|\n\d+\.)', all_text, re.DOTALL)
    if considerando_match:
        considerando_punto_1 = considerando_match.group(1).strip()
        # Buscar la frase TUPA dentro del punto 1 de CONSIDERANDO
        match = re.search(r'en el marco del procedimiento .*? \(TUPA\)', considerando_punto_1, re.IGNORECASE)
        if match:
            frase_tupa = match.group(0)

    # Procesar el texto en secciones "SE RESUELVE"
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text = page.get_text("text")
        lines = text.split('\n')

        for line in lines:
            if "SE RESUELVE:" in line:
                inside_se_resuelve = True
            if inside_se_resuelve:
                if line.startswith("Artículo "):
                    if current_article:
                        articles.append(current_article.strip())
                    current_article = line
                elif current_article:
                    if line.strip() == "":
                        articles.append(current_article.strip())
                        current_article = ""
                    else:
                        current_article += " " + line.strip()

        # Añadir el último artículo al final de la página
        if current_article:
            articles.append(current_article.strip())
            current_article = ""

    return articles, considerando_punto_1, frase_tupa

# Función para identificar los nombres estándar basados en palabras clave
def identificar_nombres_estandar(articles):
    encontrados = set()
    for nombre_estandar, keywords in procedimientos_tupa.items():
        for article in articles:
            if any(keyword in article for keyword in keywords):
                encontrados.add(nombre_estandar)
                if len(encontrados) == 2:
                    return list(encontrados)
    return list(encontrados)

# Función para identificar la embarcación pesquera y su información asociada
def identificar_embarcacion(articles):
    for article in articles:
        for matricula, data in vessels_dict.items():
            if matricula in article:
                return matricula, data['EMBARCACION'], data.get('ARMADOR', 'No identificado')
    return "No identificado", "No identificado", "No identificado"

# Crear un DataFrame para almacenar los resultados
results = []

# Procesar cada archivo PDF en la carpeta
for filename in os.listdir(folder_path):
    if filename.endswith('.pdf'):
        file_path = os.path.join(folder_path, filename)
        articles, considerando_punto_1, frase_tupa = extract_articles_from_pdf(file_path)
        nombres_estandar = identificar_nombres_estandar(articles)
        nombre_estandar_1 = nombres_estandar[0] if len(nombres_estandar) > 0 else "No identificado"
        nombre_estandar_2 = nombres_estandar[1] if len(nombres_estandar) > 1 else "No identificado"
        matricula, embarcacion, armador = identificar_embarcacion(articles)
        row = [filename, nombre_estandar_1, nombre_estandar_2, matricula, embarcacion, armador, frase_tupa, considerando_punto_1] + articles
        results.append(row)

# Determinar el número máximo de artículos para definir las columnas
max_articles = max(len(row) for row in results) - 8

# Crear nombres de columna dinámicamente
columns = ['Nombre del archivo', 'Nombre_Estandar_1', 'Nombre_Estandar_2', 'Matricula', 'Embarcacion', 'Armador', 'Frase_TUPA', 'Considerando Punto 1'] + [f'Artículo {i+1}' for i in range(max_articles)]

# Crear el DataFrame y rellenar con None donde sea necesario
df = pd.DataFrame(results, columns=columns)

# Guardar el DataFrame en un archivo Excel
output_path = '/content/tu.xlsx'
df.to_excel(output_path, index=False)

print(f"Los artículos extraídos se han guardado en {output_path}")

